# Step 4: Add a deployment pipeline
In previous four steps you implemented an automated data processing and model building pipeline. Each run of the pipeline produces a new version of the model. This notebook implements the automated model deployment step in our ML workflow.

![](img/sagemaker-mlops-project-deploy-diagram.jpg)

You can use a [SageMaker MLOps project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates.html) to provision a ready-to use model deployment CI/CD pipeline.

This template automates the deployment of models in the SageMaker model registry to SageMaker endpoints for real-time inference. This template recognizes changes in the model registry. When a new model version is registered and approved, it automatically initiates a deployment.

<div class="alert alert-info"> Make sure you using <code>Python 3</code> kernel in JupyterLab for this notebook.</div>

First, we need to install the python dependencies for this notebook

In [ ]:
%pip install jsonlines tqdm

In [ ]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep
import json
import os
from sagemaker.predictor import Predictor
import pandas as pd
from tqdm import trange
import numpy as np
sagemaker.__version__

In [ ]:
%store -r

In [ ]:
%store

In [ ]:
assert len(model_package_group_name) > 0
assert len(region) > 0
assert len(bucket_name) > 0
assert len(bucket_prefix) > 0

In [ ]:
sm = boto3.client("sagemaker")

In [ ]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("You must complete the notebook 06-sagemaker-project")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")

## Explore the project in the Studio UI

<div class="alert alert-info">Make sure you run the step 6 notebook and successfully provisioned the MLOps project and run the project pipeline at least once.</div>

Click on the links constructed by the next code cell to see the project and the model package in the Studio UI. You must see at least one version of the model registered in the model package group.

In [ ]:
# check that the project exists
project_data = sm.describe_project(ProjectName=project_name)

In [ ]:
assert project_data['ProjectStatus'] == 'CreateCompleted', 'Project must be created at this point!'

In [ ]:
# check that at least one model version is registered in the model registry
model_packages = sm.list_model_packages(
    ModelPackageGroupName=f'{project_name}-{project_id}',
    ModelApprovalStatus='PendingManualApproval')

In [ ]:
assert len(model_packages['ModelPackageSummaryList']) > 0, 'You must have at least one model version in the status PendingManualApproval'

## Working with MLOps project for model deployment
The template provisions a CodeCommit repository with configuration files to specify the model deployment steps, AWS CloudFormation templates to define endpoints as infrastructure, and seed code for testing the endpoint.

This template provides the following resources:

1. An AWS CodeCommit repository that contains template code that deploys models to endpoints in staging and production environments
2. An AWS CodePipeline pipeline that has `source`, `build`, `deploy-to-staging`, and `deploy-to-production` steps. The `source` step points to the CodeCommit repository, and the `build` step gets the code from that repository and generates CloudFormation stacks to deploy. The `deploy-to-staging` and `deploy-to-production` steps deploy the CloudFormation stacks to their respective environments. There is a manual approval step between the staging and production build steps, so that a MLOps engineer must approve the model before it is deployed to production.
3. An Amazon EventBridge rule to launch a CodePipeline pipeline execution when a model package version is approved or rejected.
4. There is also a manual approval step after the placeholder unit tests. You can implement your own tests to replace the placeholders tests.

The template also deploys an Amazon S3 bucket to store artifacts, including CodePipeline and CodeBuild artifacts, and any artifacts generated from the SageMaker pipeline runs.

The following diagram shows the architecture.

<img src="img/mlops-model-deploy.png" width="600"/>

You don't need to implement any configuration changes for the project. The model deployment pipeline works out of the box.
To start the model deployment pipeline, you must approve the model version in the model registry.

### Approve a model version
Approving a model version causes the MLOps project to launch the model deployment process. 

In the first step, the model deployment pipeline deploys the model version to a staging SageMaker real-time inference end-point.

You can approve the model version either in Studio in the Model registry or do it programmatically in the notebook. Let's do it programatically.

In [ ]:
try:
    print(model_package_group_name)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Run the step 03 notebook to create a pipeline, run the pipeline, and register a model version in the model registry")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
# list all model packages and select the latest one
model_packages = []

for p in sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Descending",
    ):
    model_packages.extend(p["ModelPackageSummaryList"])

if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group. Run a model creation pipeline first.")

print(f"There are {len(model_packages)} model versions in the {model_package_group_name} model package group")
print(f"Approve the most recent model package:")

latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

The following statement sets the `ModelApprovalStatus` for the most recent model package in the model registry to `Approved`. The model package state change launches the EventBridge rule and the rule launches the CodePipeline CI/CD pipeline with model deployment.

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

You can see the last model version in the model registry in the Studio UI changed the **Status** to `Approved`:

![](img/model-package-group-version-approval.jpg)

### Deployment pipeline execution
Upon approval of a model version in the code cell above, the model deployment CI/CD pipeline performs the following actions:
1. Create CloudFormation parameter configuration files with stating and prod parameters for CloudFormation templates with SageMaker endpoint IaC
1. Create a SageMaker real-time inference endpoint with the name `<PROJECT-NAME>-staging` in the current account
1. Run the test script on the staging endpoint
1. Wait until the test result is manually approved in [AWS CodePipeline console](https://console.aws.amazon.com/codesuite/codepipeline)
1. Create a SageMaker endpoint with the name `<PROJECT-NAME>-prod` in the current account

Wait about 10-15 minutes until the pipeline finishes deployment of the staging endpoint. You can see the status of the endpoint in the Studio UI in **Deployments** > **Endpoints**:

![](img/sagemaker-mlops-deploy-endpoint-status.jpg)

After the endpoint status changed from `Creating` to `InService`, the staging endpoint is fully operational. You can launch the model deployment process to the production stage by manually approving the **DeployStaging** stage of the CodePipeline pipeline. In the next section you approve the model deployment and launch the second stage of the deployment into a production endpoint.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">Wait until staging endpoint status changes to InService, then continue with the following code cells.
    </p>
</div>

# Testing the SageMaker endpoint
After a successful deployment to a SageMaker endpoint in staging, let's verify the endpoint by running some inferences through it.
When it comes to serving your model in an endpoint, Sagemaker offers many different options:

## SageMaker Endpoint
SageMaker provides different options for your inference use cases, giving you choice over the technical breadth and depth of your deployments:

* **Deploying a model to an endpoint.** When deploying your model, consider the following options:
   + [Real-time inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html). Real-time inference is ideal for inference workloads where you have interactive, low latency requirements.
   + [Deploy models with Amazon SageMaker Serverless Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html). Use Serverless Inference to deploy models without configuring or managing any of the underlying infrastructure. This option is ideal for workloads which have idle periods between traffic spurts and can tolerate cold starts.
   + [Asynchronous inference](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html). queues incoming requests and processes them asynchronously. This option is ideal for requests with large payload sizes (up to 1GB), long processing times (up toAsynchronous Inference one hour), and near real-time latency requirements

* **Cost optimization**. To optimize your inference costs, consider the following options:

   + [Automatically Scale Amazon SageMaker Models](https://docs.aws.amazon.com/sagemaker/latest/dg/endpoint-auto-scaling.html). Use autoscaling to dynamically adjust the compute resources for your endpoints based on incoming traffic patterns, which helps you optimize costs by only paying for the resources you're using at a given time.
 
The following diagram provides an overview of all the deployment options in SageMaker:

![](img/sagemaker-deployment-modes.jpg)

## Real time Inference from a SageMaker endpoint
To demonstrate the inference capabilities, we'll explore both realtime and batch transform in this notebook. We'll use the staging endpoint for our test so that we know the endpoint works as expected before deploying to production environment. 

In [ ]:
# List all deployed real-time endpoints
endpoints = sm.list_endpoints(StatusEquals="InService")["Endpoints"]

if not len(endpoints):
    print("There are no deployed active endpoints. You must have at least one endpoint. Run the previous cell in this notebook to deploy an endpoint")
else:
    print(f"Found {len(endpoints)} active inference endpoint(s):\n")
    
    for i, endpoint in enumerate(endpoints, 1):
        print(f"{i}. Endpoint Name: {endpoint['EndpointName']}")
        print(f"   Status: {endpoint['EndpointStatus']}")
        print(f"   Creation Time: {endpoint['CreationTime']}")
        print(f"   Last Modified: {endpoint['LastModifiedTime']}")
        print("---")
    
    # If you still need to select one endpoint for further operations, 
    # you can use the first one or add selection logic:
    endpoint_name = endpoints[0]['EndpointName']
    print(f"\nUsing endpoint '{endpoint_name}' for subsequent operations.")


> [!NOTE]
> If you are running this workshop as an instructor led training, the `endpoint_name` is set for you so there is nothing for you to do. Otherwise, simply update the endpoint_name with the appropriate endpoint_name for the test.

### Define helper functions
Define some helper functions with code snippets that you're going to use throughout this notebook.

In [ ]:
# Send data to the endpoint
def realtime_prediction(predictor, data):
    l = len(data)
    for i in trange(l):
        data_arr = [float(np_float) for np_float in data.iloc[i].values ]
        predictions = np.array(predictor.predict(data_arr), dtype=float).squeeze()
        print(predictions)

def download_from_s3(s3_client, local_file_path, bucket_name, s3_file_path):
    try:
        # Download the file
        s3_client.download_file(bucket_name, s3_file_path, local_file_path)
        print(f"File downloaded successfully to {local_file_path}")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            print(f"An error occurred: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

def upload_to_s3(s3_client, local_file_path, bucket_name, s3_file_path=None):
    # If S3 file path is not specified, use the basename of the local file
    if s3_file_path is None:
        s3_file_path = os.path.basename(local_file_path)

    try:
        # Upload the file
        s3_client.upload_file(local_file_path, bucket_name, s3_file_path)
        print(f"File {local_file_path} uploaded successfully to {bucket_name}/{s3_file_path}")
        return True
    except ClientError as e:
        print(f"ClientError: {e}")
        return False
    except FileNotFoundError:
        print(f"The file {local_file_path} was not found")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False
        
def write_params(s3_client, step_name, params, notebook_param_s3_bucket_prefix):
    local_file_path = f"{step_name}.json"
    with open(local_file_path, "w") as f:
        f.write(json.dumps(params))
    base_local_file_path = os.path.basename(local_file_path)
    bucket_name = notebook_param_s3_bucket_prefix.split("/")[2] # Format: s3://<bucket_name>/..
    s3_file_path = os.path.join("/".join(notebook_param_s3_bucket_prefix.split("/")[3:]), base_local_file_path)
    upload_to_s3(s3_client, local_file_path, bucket_name, s3_file_path)
    
def read_params(s3_client, notebook_param_s3_bucket_prefix, step_name):
    local_file_path = f"{step_name}.json"
    base_local_file_path = os.path.basename(local_file_path)
    bucket_name = notebook_param_s3_bucket_prefix.split("/")[2] # Format: s3://<bucket_name>/..
    s3_file_path = os.path.join("/".join(notebook_param_s3_bucket_prefix.split("/")[3:]),  base_local_file_path)
    downloaded = download_from_s3(s3_client, local_file_path, bucket_name, s3_file_path)
    with open(local_file_path, "r") as f:
        data = f.read()
        params = json.loads(data)
    return params

### Generate realtime prediction based on test data
In the following cell, we'll run some inferences in realtime using the test data captured in the previous notebook.

In [ ]:
# Create a predictor class for the endpoint
predictor = Predictor(
    endpoint_name=endpoint_name, 
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer()
)

In [02-preprocess.ipynb](02-preprocess.ipynb) we divided the dataset into training, validation and test dataset. For this lab, we'll use the test dataset for running inferences against the deployed endpoint. 

In [ ]:
preprocess_step_name = "02-preprocess"
s3_client = boto3.client("s3", region_name=region)
notebook_param_s3_bucket_prefix=f"s3://{bucket_name}/{bucket_prefix}/params"
preprocess_step_params = read_params(s3_client, notebook_param_s3_bucket_prefix, preprocess_step_name)

In [ ]:
local_test_x_file_path = "test_x.csv"
local_test_y_file_path = "test_y.csv"
s3_test_x_data = preprocess_step_params["test_x_data"]
s3_test_y_data = preprocess_step_params["test_y_data"]

# Download the test_x.csv and test_y.csv file from S3
bucket_name = s3_test_x_data.split("/")[2]
s3_test_x_data_key = "/".join(s3_test_x_data.split("/")[3:])
s3_test_y_data_key = "/".join(s3_test_y_data.split("/")[3:])
download_from_s3(s3_client, local_test_x_file_path, bucket_name, s3_test_x_data_key)
download_from_s3(s3_client, local_test_y_file_path, bucket_name, s3_test_y_data_key)

In [ ]:
# Set the number of data vectors from the test dataset sent to the inference endpoint as batch
number_of_vectors = 10
test_x = pd.read_csv("test_x.csv", header=None).sample(number_of_vectors)

# Select only the first 10 features (columns 0-9) to match the trained model
test_x = test_x.iloc[:, :10]

In [ ]:
# Prints the output to see the response payload
print(f"Test data shape after feature selection: {test_x.shape}")
test_x.head()

Rename the column names for identifying the feature attributes

In [ ]:
# test_x will have 10 columns
test_x.columns = [f'_c{i}' for i in range(len(test_x.columns))]

Run prediction using the realtime endpoint deployed in the previous step

In [ ]:
realtime_prediction(predictor, test_x)

# Batch Transform
SageMaker offers [batch transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) to optimize inference workloads for the following  scenarios:

* Preprocess datasets to remove noise or bias that interferes with training or inference from your dataset.
* Get inferences from large datasets.
* Run inference when you don't need a persistent endpoint.
* Associate input records with inferences to help with the interpretation of results.

Functionally, batch transform uses the same mechanics as real-time hosting to generate predictions. It requires a web server that takes in HTTP POST requests a single observation, or mini-batch, at a time. However, unlike real-time hosted endpoints which have persistent hardware (instances stay running until you shut them down), batch transform clusters are torn down when the job completes.

To demonstrate the capability, we'll run a batch transform job on the same dataset that we used for realtime inference previously.

In [ ]:
# Download and preprocess the test data to use only first 10 features
import pandas as pd

# Download the original test data
local_test_x_batch_file = "test_x_batch_full.csv"
download_from_s3(s3_client, local_test_x_batch_file, bucket_name, s3_test_x_data_key)

# Load and select only first 10 features
test_x_full = pd.read_csv(local_test_x_batch_file, header=None)
test_x_batch = test_x_full.iloc[:, :10]  # Select only first 10 features

print(f"Original batch data shape: {test_x_full.shape}")
print(f"Modified batch data shape: {test_x_batch.shape}")

# Save the modified test data locally
local_test_x_batch_processed = "test_x_batch_processed.csv"
test_x_batch.to_csv(local_test_x_batch_processed, header=False, index=False)

# Upload the processed data to S3
s3_test_x_batch_processed_key = f"{bucket_prefix}/test/test_x_batch_processed.csv"
s3_test_x_batch_processed_path = f"s3://{bucket_name}/{s3_test_x_batch_processed_key}"

s3_client.upload_file(
    local_test_x_batch_processed, 
    bucket_name, 
    s3_test_x_batch_processed_key
)

print(f"Processed batch data uploaded to: {s3_test_x_batch_processed_path}")

In [ ]:
from sagemaker.transformer import Transformer
from sagemaker.inputs import TransformInput

First we'll get the model_name from the deployed sagemaker endpoint. 

In [ ]:
response = sm.describe_endpoint(
    EndpointName=endpoint_name
)
endpoint_config_name = response["EndpointConfigName"]
response = sm.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
model_name = response['ProductionVariants'][0]['ModelName']

Define batch_transform variables to use for running a batch tranform job.

In [ ]:
batch_transform_instance_type = "ml.m5.large"
batch_transform_output_path = f"s3://{bucket_name}/{bucket_prefix}/transform"
sagemaker_session = sagemaker.Session()

In [ ]:
# create the transform step
transformer = Transformer(
        model_name=model_name,
        instance_type=batch_transform_instance_type,
        instance_count=1,
        accept="text/csv",
        assemble_with="Line",
        output_path=batch_transform_output_path,
        sagemaker_session=sagemaker_session,
        base_transform_job_name=f"player-churn-model-batch-transform",
    )

Trigger a batch tranform job using the SageMaker python SDK

In [ ]:
# Use the processed data with only 10 features instead of original data
transformer.transform(    
        data=s3_test_x_batch_processed_path,  # Changed from s3_test_x_data
        content_type="text/csv",
        split_type="Line", 
        join_source="Input"
    )

Download the inference results from the S3 bucket.

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./

Let's take a look at the payload. The predicted value 

> [!NOTE]
> The results above shows the response payload that contains the input data and the predictions in CSV format. The payload contains both the input and the predicted label. In addition to the default output structure, you can also customize the way batch transform constructs the output. Please refer to this [link](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html#batch-transform-data-processing-examples) to learn more about these customizations. 

In [ ]:
!head -10 test_x_batch_processed.csv.out

### Deploy the model version to production
Now that we've validated the staging endpoint, assuming we are happy with the results. Next, we'll continue the deployment pipeline to production deployment. 

Let's construct a CodePipeline approval link. 

If you used the option 1 `boto3` to create an MLOps project, the `project_name` and `project_id` are set automatically. You can run the following code cell to print the values. If you followed the UI instructions to create a project, you must set the `project_name` manually.

In [ ]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("++++++++++++++++++++++++++++++++++++++")
    print("You must set the project_name manually")
    print("++++++++++++++++++++++++++++++++++++++")

In [ ]:
# Set to the model deployment project name if you didn't use boto3-based deployment
# project_name = "<PROJECT_NAME>"

# Get project id
project_id = sm.describe_project(ProjectName=project_name)['ProjectId']

# Construct the CodePipline pipeline name
code_pipeline_name = f"sagemaker-{project_name}-{project_id}-modeldeploy"

In [ ]:
 # approve the latest model version
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

In [ ]:
from IPython.display import HTML
# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">AWS CodePipeline</a></b>'.format(
            code_pipeline_name, region)
    )
)

Click on the link ^^^ above ^^^ to open the CodePipeline console with the pipeline execution workflow.

In the **DeployStaging stage**, choose **Review** on the **ApproveDeployment** step. Note, you might wait until `TestStaging` step completes with `Succeeded` status. 

![](img/deploy-staging-review.png)

In the **Review** dialog box, select **Approve** and choose **Submit**:

![](img/approve-deployment.png)

Approving the **DeployStaging** stage causes the deployment pipeline to continue and to deploy the model to the production endpoint. To view the endpoints, choose the **Deployments** > **Endpoints** in Studio UI.

As your CI/CD deployment pipeline continues, you see the production endpoint in status `Creating` along with the previously deployed staging endpoint in status `InService`:

![](img/endpoint-prod-creating.png)

After `10-15` min the deployment is completed and both endpoints are in status `InService`.

Navigate to the Studio and choose **Deployments** > **Projects**. In the Project pane select `model-deploy-<TIMESTAMP>` project. In the project details pane select **Endpoints**. You see that both endpoints, `staging` and `prod`, are visible in the deployment project because the project and the endpoints are connected via the metadata:

![](img/project-endpoints.png)


## Summary
In this notebook you implemented an automated CI/CD deployment pipeline with the following features:
- use CloudFormation IaC templates for SageMaker real-time inference endpoint deployment
- model approval in the model registry launches the model deployment pipeline
- model deployment pipeline contains two stages, staging and production with automated tests for the staging endpoint and manual approval for the production deployment

---

## Clean-up
<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">
    If you're going to run the step 6 notebook (Data and Model Quality Monitoring), you need to keep at least one of the endpoints. If you finish the workshop here and don't run the step 6 notebook, navigate to the <b>clean-up notebook (99-clean-up.ipynb)</b> and follow the clean-up instructions to avoid charges in your AWS account.
    <br>
    <br>
    You don't need to run the clean-up if you're using an AWS-provided AWS account.
    </p>
</div>

## Further development ideas for your real-world projects
- Add end-to-end data encryption using AWS KMS keys
- Create a [custom SageMaker project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) for model deployment to cover your specific project requirements
- Add [multi-account model deployment](https://aws.amazon.com/blogs/machine-learning/multi-account-model-deployment-with-amazon-sagemaker-pipelines/) to your ML workflow
- Add automated model tests to the placeholder in the CodePipeline pipeline
- Use [Amazon SageMaker Inference Recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html) to run automated load tests for your inference endpoints and to select the best instance type and configuration

## Additional resources
- [Deploy a Machine Learning Model to a Real-Time Inference Endpoint](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)
- [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [Amazon SageMaker Pipelines lab in SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Amazon SageMaker secure MLOps](https://github.com/aws-samples/amazon-sagemaker-secure-mlops)
- [Testing approaches for Amazon SageMaker ML models](https://aws.amazon.com/blogs/machine-learning/testing-approaches-for-amazon-sagemaker-ml-models/)
- [Model hosting patterns in Amazon SageMaker blog series](https://aws.amazon.com/blogs/machine-learning/model-hosting-patterns-in-amazon-sagemaker-part-1-common-design-patterns-for-building-ml-applications-on-amazon-sagemaker/)
- [Take advantage of advanced deployment strategies using Amazon SageMaker deployment guardrails](https://aws.amazon.com/blogs/machine-learning/take-advantage-of-advanced-deployment-strategies-using-amazon-sagemaker-deployment-guardrails/)
- [MLOps deployment best practices for real-time inference model serving endpoints with Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/mlops-deployment-best-practices-for-real-time-inference-model-serving-endpoints-with-amazon-sagemaker/)

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>